# Version 2 of DDR Spectrogram

# We Will Train the Steps from Spectrograms of MP3 Files

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import re

In [ ]:
#Get the list of all of the step files
step_files = list(Path("C:/Users/brent/Desktop/StepMania 5").rglob("*.[dD][wW][iI]"))

#Get the list of all of the step files
song_files = list(Path("C:/Users/brent/Desktop/StepMania 5").rglob("*.[mM][pP][3]"))

In [ ]:
#There are not equal amounts of mp3s and matching step files
print(len(song_files), len(step_files))

### We modify the code below to capture the gap and the bpm

In [ ]:
def process_song(path, title):
    #Open File
    text_file = open(path, "r")
    lines = text_file.readlines()
    text_file.close()
    
    #Combine all text into single line
    song = "".join(lines)
    
    #Remove newline characters
    song = re.sub('\n', '', song)
    
    #Split on semicolon and then add the semicolons back into the respective lines
    song = song.split(';')
    song = [line+';' for line in song][:-1]
    
    #Remove lines that start with 2 // (some files had this for some reason)
    song = [line for line in song if (line.find('//') == -1)]
    
    #Create a dataframe of the song
    df = pd.DataFrame()
    df[title] = song
    return df

def pull_all_step_patterns(song, row, path):
    song = song[row].str.split(":", n = 3, expand = True)
    
    #Get BPM
    bpm = song[song[0] == "#BPM"].iloc[0,1]
    #Remove ;
    bpm = float(bpm[:-1])
    
    #Get Gap
    gap = song[song[0] == "#GAP"].iloc[0,1]
    #Remove ;
    gap = float(gap[:-1])
    
    song = song[song[0].isin(["#SINGLE","#SOLO"])]
    
    song['4'] = bpm
    song['5'] = gap
    song['6'] = path
    
    return song

    
def join_all_step_patterns(step_files):
    songs = pd.DataFrame()
    for row, path in enumerate(step_files):
        df = process_song(path, row)
        df = pull_all_step_patterns(df, row, path)
        
        #songs = pd.merge(songs, df, left_index=True, right_index=True, how="outer")
        songs = pd.concat([songs,df])
    
    return songs

In [ ]:
songs = join_all_step_patterns(step_files)

# Iterate through step patterns and Specify location of mp3

In [ ]:
songs = songs.reset_index()
songs.drop(['index'], axis=1, inplace=True)

In [ ]:
#Check that all step files have matching mp3 files
#Create a dataframe that has both the step file path as well as the mp3 path
for index, row in songs.iterrows():
    #Get location of each dwi file and then determine location of mp3 file
    steppath = songs.iloc[index, 6]
    songpath = Path(steppath.parent.as_posix() + '/' + steppath.stem + '.mp3')
    
    if songpath in song_files:
        songs.loc[index, '7'] = songpath
    else:
        songs.loc[index, '7'] is None      

### Now we remove the step files that don't have interpreted mp3's

In [ ]:
songs = songs[~songs['7'].isnull()]

In [ ]:
songs = songs.reset_index()
songs.drop(['index'], axis=1, inplace=True)

# Create Spectrograms for Matching Songs

In [ ]:
def mp3_to_spectrogram(path, gap):
    #offset should be duration of gap specifided in .DWI FIle
    #Gap is in milliseconds while offset is in seconds

    #Choose the file to create the graphics
    filename = path

    #y, sr = librosa.load(filename,offset=30, duration=12.0)
    y, sr = librosa.load(filename, offset=gap/1000)
    onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
    librosa.frames_to_time(onset_frames, sr=sr)

    o_env = librosa.onset.onset_strength(y, sr=sr)
    times = librosa.frames_to_time(np.arange(len(o_env)), sr=sr)
    onset_frames = librosa.onset.onset_detect(onset_envelope=o_env, sr=sr)
    D = np.abs(librosa.stft(y))

    #Create the spectrogram
    plt.clf()
    plt.axis('off')
    librosa.display.specshow(librosa.amplitude_to_db(D, ref=np.max),x_axis='time', y_axis='log')
    plt.savefig(Path('image')/(filename.stem+'_1.png'))

    #Create a beat offset graph
    plt.clf()
    plt.axis('off')
    plt.plot(times, o_env)
    plt.savefig(Path('image')/(filename.stem+'_2.png'))

#### This next block creates about 1500 spectrograms and takes a long time.  On the order of hours

In [ ]:
#Iterate through each row of the dataframe and create both versions of the spectrogram.
for index, row in songs.iterrows():
    songpath = songs.iloc[index, 7]
    gap = songs.iloc[index, 5]
    
    imagepath1 = Path('image' + '/' + songpath.stem + '_1.png')
    imagepath2 = Path('image' + '/' + songpath.stem + '_2.png')

    songs.loc[index, '8'] = imagepath1
    songs.loc[index, '9'] = imagepath2
    
    #Generate the actual spectrograms
    #Toggled Off because it has been previously run
    #mp3_to_spectrogram(songpath, gap)

# Create the final file that we will use for our model

In [ ]:
songs.columns = ['mode','label','difficulty','text','bpm','gap','step_path','mp3_path','spectrogram_path','spectrogram_path2']

In [ ]:
#songs = songs[[1,3]]
#songs.columns = ['label','text']

#Split the song into characters with semicolons
songs['text'] = songs['text'].apply(lambda x: ";".join(x))

#Remove the trailing semicolon as we can add it back in when we are done predicting songs
songs['text'] = songs['text'].apply(lambda x: x[:-2])

In [ ]:
songs.to_csv("spectrogram.csv", index=False)

# We Will Create our Data Bunch from the Spectrogram Images

In [ ]:
import pandas as pd

In [ ]:
songs = pd.read_csv('spectrogram.csv', index_col=False)

In [ ]:
from fastai.vision import *

### Use multi category label through use of ';' as a delimiter

In [ ]:
#Create our databunch
#We choose not to use any transforms as the images
#were created in a uniform fashion from mp3s
data = (ImageList.from_df(df=songs, path='.', cols = ['spectrogram_path'])
        .split_by_rand_pct(valid_pct=0.2)
        .label_from_df(cols=['text'],label_delim=';')
        .transform(([],[]))
        .databunch(bs=4))

In [ ]:
learn = cnn_learner(data, models.resnet152, metrics=[mean_absolute_error])

In [ ]:
learn.lr_find()
learn.recorder.plot(skip_end=15)

In [ ]:
learn.fit_one_cycle(100,1e-3)

In [ ]:
learn.save('spectrogram_multicategory')

In [ ]:
learn = cnn_learner(data, models.resnet152, metrics=[mean_absolute_error])

In [ ]:
learn.load('spectrogram_multicategory')
pass

In [ ]:
data.x[1000]

In [ ]:
data.y[100]

In [ ]:
 data.train_ds.y.items[0]

In [ ]:
data.train_ds.y.__class__

# Using Regular Category Labels

In [ ]:
#Create our databunch
#We choose not to use any transforms as the images
#were created in a uniform fashion from mp3s
data = (ImageList.from_df(df=songs, path='.', cols = ['spectrogram_path'])
        .split_by_rand_pct(valid_pct=0)
        .label_from_df(cols=['text'])
        .transform(([],[]))
        .databunch(bs=8))

In [ ]:
learn = cnn_learner(data, models.resnet152, metrics=[accuracy])

In [ ]:
learn.lr_find()
learn.recorder.plot(skip_end=15)

In [ ]:
learn.fit_one_cycle(20,1e-4)

In [ ]:
learn.save('spectrogram2')

In [ ]:
learn.fit_one_cycle(20,1e-4)

In [ ]:
learn.save('spectrogram2')

In [ ]:
learn.fit_one_cycle(20,1e-4)

In [ ]:
learn.save('spectrogram2')

In [ ]:
learn.fit_one_cycle(20,1e-4)

In [ ]:
learn.fit_one_cycle(20,1e-4)

In [ ]:
learn.save('spectrogram2')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(100,1e-5)

In [ ]:
learn.save('spectrogram2')

# We Now Have Our Trained Model and Area Ready for Prediction

In [ ]:
learn.load('spectrogram2')
pass

In [ ]:
data.x[1000]

In [ ]:
learn.predict(data.x[1000])